<a href="https://colab.research.google.com/github/coughlinjennie/data71200/blob/main/Data71200_project3_Coughlin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Load data and libraries

In [ ]:
#Load libraries

import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import requests
import io
import matplotlib.pyplot as plt

!pip install -U scikit-learn==1.4

import six
import joblib
import sys
sys.modules['sklearn.externals.six'] = six
sys.modules['sklearn.externals.joblib'] = joblib
!pip install mglearn
import mglearn

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load data

#Import the data, sourced from Kaggle and stored in my GitHub
url = "https://raw.githubusercontent.com/coughlinjennie/data71200/main/projects/nyhousing.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
#Load the data

housing_master = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [ ]:
# Delete all rows where column 'TYPE' has certain values
indexType = housing_master[ (housing_master['TYPE'] == "For sale") | (housing_master['TYPE'] == "Contingent") | (housing_master['TYPE'] == "Land for sale") | (housing_master['TYPE'] == "Foreclosure") | (housing_master['TYPE'] == "Pending") | (housing_master['TYPE'] == "Coming Soon") | (housing_master['TYPE'] == "Mobile house for sale") ].index
housing_master.drop(indexType , inplace=True)

#Set the labels on TYPE

housing_label = housing_master["TYPE"]

#Set the data
housing = housing_master.drop("TYPE", axis=1)

#Divide the data into training and testing sets
from sklearn.model_selection import train_test_split

housing_train, housing_test, housing_label_train, housing_label_test = train_test_split(housing, housing_label, test_size=0.3, stratify=housing_label, random_state=42)

# Create a list of redundant column names to drop from the training data only
to_drop = ["LONGITUDE", "LATITUDE", "ADDRESS", "ADMINISTRATIVE_AREA_LEVEL_2", "LOCALITY", "SUBLOCALITY", "FORMATTED_ADDRESS", "MAIN_ADDRESS", "STATE", "STREET_NAME","LONG_NAME","BROKERTITLE"]

# Drop those columns from the dataset
housing_subset = housing_train.drop(to_drop, axis = 1)
h_test_subset = housing_test.drop(to_drop, axis = 1)

#Drop all properties that sold for more than $1B from training data only

housing_clean = housing_subset[housing_subset['PRICE'] <= 100000000]
label_train_clean = housing_label_train[housing_subset['PRICE'] <= 100000000]


In [ ]:
#Scale features as needed

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
housing_scaled = scaler.fit_transform(housing_clean)


#Step 2: PCA for feature selection

In [ ]:
# keep the components that explain 95% of the variance in the data
pca = PCA(n_components=0.95)
# fit PCA model to data
housing_pca = pca.fit_transform(housing_scaled)


In [ ]:
#Run the Decision Tree model from Project 2 again with PCA data
model3 = DecisionTreeClassifier(random_state=0).fit(housing_pca, label_train_clean)

print("Accuracy on training set: {:.2f}".format(model3.score(housing_pca, label_train_clean)))


In [ ]:
#Evaluate model performance vs. original

model = DecisionTreeClassifier(random_state=0).fit(housing_scaled, label_train_clean)
print("Accuracy on training set: {:.2f}".format(model.score(housing_scaled, label_train_clean)))



#Step 3: Apply Clustering to Data and Visualize Output

##K-means clustering

In [ ]:
#Run the model with PCA applied

In [ ]:
#Calculate the ARI

In [ ]:
#Calculate the Silhouette Configuration

In [ ]:
#Run the model without PCA applied

In [ ]:
#Calculate the ARI

In [ ]:
#Calculate the Silhouette Configuration

##Agglomerate/hierarchical

In [ ]:
#Run the model with PCA applied

In [ ]:
#Run the model without PCA applied

##DBSCAN

In [ ]:
#Run the model with PCA applied

In [ ]:
#Run the model without PCA applied